In [2]:
import numpy as np
import pandas as pd

import psi4

from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.metrics import classification_report



import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.tri as tri

import numpy_html

import time

from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline


psi4.set_options({'guess' : 'sad',
                  'dft_spherical_points' : 590, 
                  'dft_radial_points' : 99,   
                  'MAXITER'   :  2000})


In [3]:
diatoms = pd.read_csv("diatoms_raw_full.csv")

atoms={"H":1,"He":2, "Li":3, "Be":4, "B":5, "C":6, "N":7, "O":8, "F":9,
         "Ne":10, "Na":11, "Mg":12, "Al":13, "Si":14, "P":15, "S":16, 
         "Cl":17, "Ar":18, "K":19, "Ca":20, "Sc":21, "Ti":22, "V":23,
         "Cr":24, "Mn":25, "Fe":26, "Co":27, "Ni":28, "Cu":29, "Zn":30,
         "Ga":31, "Ge":32, "As":33, "Se":34, "Br":35, "Kr":36} 

atom_list1 = ['H', 'Li', 'Li', 'Be', 'B', 'B', 'C', 'N', 'B',  'C', 'N', 'O', 'Li', 'Be', 'B', 'C', 'N', 'O', 'H', 'Li',  'B', 'C', 'N', 'F', 'F', 'Na', 'Na', 'Na', 'Na', 'Mg', 'Mg', 'Mg', 'Al', 'Al', 'Al', 'Al', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'P', 'P', 'P', 'P', 'Si', 'P',  'H', 'Li', 'Be', 'B', 'C', 'N', 'S', 'S', 'Na', 'Mg', 'Al', 'Si', 'S', 'H',   'Li', 'B', 'C'  ,'N' , 'Cl', 'Cl', 'Na',   'Al', 'Si', 'P', 'Cl' ]  
atom_list2 = ['H', 'H',  'Li', 'H', 'H', 'B',  'C', 'H', 'N', 'N',  'N', 'H', 'O',  'O', 'O',  'O', 'O', 'O', 'F',  'F',  'F', 'F', 'F', 'O', 'F', 'H', 'Li',  'F',  'Na', 'H', 'O',  'Mg' , 'H',  'C',  'F', 'Al' , 'H', 'C',   'N' , 'O' ,  'F', 'Si', 'H', 'N', 'O', 'F' , 'P', 'P',  'S', 'S',  'S',  'S', 'S', 'S', 'O',  'F',  'S', 'S', 'S' , 'S',  'S',  'Cl', 'Cl', 'Cl', 'Cl', 'Cl', 'O',  'F',  'Cl',  'Cl', 'Cl',  'Cl', 'Cl']


atoms={"H":1,"He":2, "Li":3, "Be":4, "B":5, "C":6, "N":7, "O":8, "F":9,
         "Ne":10, "Na":11, "Mg":12, "Al":13, "Si":14, "P":15, "S":16, 
         "Cl":17, "Ar":18, "K":19, "Ca":20, "Sc":21, "Ti":22, "V":23,
         "Cr":24, "Mn":25, "Fe":26, "Co":27, "Ni":28, "Cu":29, "Zn":30,
         "Ga":31, "Ge":32, "As":33, "Se":34, "Br":35, "Kr":36} 

assert len(atom_list1) == len(atom_list2)

assert len(atom_list1) == len(atom_list2)

In [4]:
#Generates coulomb matrix for ML analysis

coulomb_matrix = np.zeros((diatoms.shape[0],2,2))
coulomb_vector = np.zeros((diatoms.shape[0],2))
coulomb_vector_long = np.zeros((diatoms.shape[0],3))

coulomb_labels_correction = np.array(-diatoms.loc[:, "difference"])
coulomb_labels_ccsdt = np.array(-diatoms.loc[:, "ccsdt"])
coulomb_labels_hf = np.array(-diatoms.loc[:, "ccsdt"])


for matrix in range(diatoms.shape[0]):
    
    if atoms[diatoms.loc[matrix,"atom1"]] >= atoms[diatoms.loc[matrix,"atom2"]]:
    
        #fill in the main diagonal
        coulomb_matrix[matrix,0,0] =  0.5*atoms[diatoms.loc[matrix,"atom1"]]**2.4
        coulomb_matrix[matrix,1,1] =  0.5*atoms[diatoms.loc[matrix,"atom2"]]**2.4
        
    elif diatoms.loc[matrix,"atom2"] >  diatoms.loc[matrix,"atom1"]:
        
        #fill in the main diagonal
        coulomb_matrix[matrix,1,1] =  0.5*atoms[diatoms.loc[matrix,"atom1"]]**2.4
        coulomb_matrix[matrix,0,0] =  0.5*atoms[diatoms.loc[matrix,"atom2"]]**2.4
        
    #fill in the non-diagonal elements
    coulomb_matrix[matrix,0,1] =  atoms[diatoms.loc[matrix,"atom1"]] * atoms[diatoms.loc[matrix,"atom2"]] / diatoms.loc[matrix,"r"]
    coulomb_matrix[matrix,1,0] =  atoms[diatoms.loc[matrix,"atom1"]] * atoms[diatoms.loc[matrix,"atom2"]] / diatoms.loc[matrix,"r"]
    
    coulomb_vector_long[matrix,0] =  coulomb_matrix[matrix,0,0]
    coulomb_vector_long[matrix,1] =  coulomb_matrix[matrix,1,1]
    coulomb_vector_long[matrix,2] =  coulomb_matrix[matrix,0,1]
    
    
    #Get eigenvalues and eigenvectors
    values, vectors = np.linalg.eig(coulomb_matrix[matrix,:,:])
    
    #Get an array from each matrix
    coulomb_vector[matrix,:] = values

## RBF Kernel / Gaussian Kernel

$$
k(x,y) = exp(-\gamma ||x-2||^2)
$$

where $\gamma = 1/ 2\sigma^2$

In [5]:
pd_coulomb_vector = pd.DataFrame(coulomb_vector_long)
pd_ccsdt      = diatoms.loc[:,"ccsdt"]
pd_difference = diatoms.loc[:, "difference"]

In [ ]:
#Grid Search for Thr Gaussian Kernel. Performs nested and non-nested scores. Nested is very expensive. 

#def optimize_KKR_hyper(features, labels, folds):
#
#    alphas = np.linspace(0.000010, 1, 1000)
#    optimizer = {"alphas":[], "scores":[]}
#    
#    for i, alpha_i in enumerate(alphas):
#            
#        #optimizer["sigmas"].append(sigma_i)
#        optimizer["alphas"].append(alpha_i)
#
#        X_train, X_test, y_train, y_test = train_test_split(features, labels)
#        KKR = KernelRidge(alpha=alpha_i, kernel='rbf', gamma=None)
#        KKR.fit(X_train, y_train)
#        y_out = pd.DataFrame(KKR.predict(X_test), index=X_test.index)
#
#        score = KKR.score(X_test, y_test)
#
#        optimizer["scores"].append(score)
#        #optimizer[i,j] = score
#            
#            
#    return optimizer

parameters = {"alpha" : np.linspace(0, 1.1e-5, 10),
              "gamma" : np.linspace(1e-1, 1e-7, 10)}


gaussian = KernelRidge(kernel="rbf")
nested_scores = np.zeros(numberoftrials)
non_nested_scores = np.zeros(numberoftrials)


inner_cv = KFold(n_splits=10, shuffle=True, random_state=i)
    #outer_cv = KFold(n_splits=10, shuffle=True, random_state=i)
    
kr = GridSearchCV(estimator=gaussian,param_grid=parameters, cv=inner_cv)
kr.fit(pd_coulomb_vector, pd_difference)
scores = kr.best_score_

#    nested_score = cross_val_score(kr, X=pd_coulomb_vector, y=pd_difference, cv=outer_cv)
#    nested_scores[i] = nested_score.mean()

#score_difference = non_nested_scores - nested_scores
#print("Average difference of {:6f} with std. dev. of {:6f}."
#      .format(score_difference.mean(), score_difference.std()))


#X_train, X_test, y_train, y_test = train_test_split(pd_coulomb_vector, pd_ccsdt)
#kr = GridSearchCV(kernel, cv=2, param_grid=param_grid)
#kr.fit(X_train, y_train)


In [36]:
#Turn results into a dataframe
results = pd.DataFrame(kr.cv_results_)
#Provides the mean of the scores
print('Best score', kr.best_score_)
#Provides the best hyperparameters for the dataset given
print('Best parameters',kr.best_params_ )

## Illustrations for the proposal

In [72]:
#######################
#Plot of Histogram. Depends on atoms dictionary and diatomics DF and individual calculations on HCL
#######################


atoms = pd.read_csv("atoms.csv")
atoms.set_index('z', inplace=True)

h_ccsdt = atoms.loc[0, "ccsdt"]
h_hf    = atoms.loc[0, "hf"]
h_dft   = atoms.loc[0, "dft"]

cl_ccsdt = atoms.loc[14, "ccsdt"]
cl_hf    = atoms.loc[14, "hf"]
cl_dft   = atoms.loc[14, "dft"]
  
ccsdt = h_ccsdt + h_ccsdt
hf =  h_hf + h_hf
dft = h_dft + h_dft

fig, ax = plt.subplots(nrows=1, ncols=2,figsize=((15,5)), dpi=300)


p1 = ax[0]


p1.tick_params(axis='both', which='major', labelsize=12)
p1.plot([0,10],[0,0], color = 'grey', lw=6, ls=':',zorder=1)


p1.set_xlabel("R", fontsize=20)
p1.set_ylabel("Binding Energy (hartree)", fontsize=20)

i=2
j=100

p1.plot(diatomic_energies["r"][i:j],diatomic_energies["ccsdt"][i:j] - ccsdt, label='CCSD(T)', 
         color='#d9485d', lw=8 )

p1.plot(diatomic_energies["r"][i:j],diatomic_energies["hf"][i:j]    - hf, label='RHF',       
         color='#00a2ff', lw=8 , zorder=4)

p1.plot(diatomic_energies["r"][i:j],diatomic_energies["dft"][i:j]  - dft, label='LDA',      
         color='#ff01ff', lw=8 )


p1.set_xlim([0.6, 10])

p1.legend(fontsize=20)

###############   Differences    #####################

p2 = ax[1]

p2.tick_params(axis='both', which='major', labelsize=12)
p2.plot([0,10],[0,0], color = 'grey', lw=6, ls=':',zorder=1)

p2.set_xlabel("R", fontsize=20)
p2.set_ylabel("CCSD(T) $\Delta E$ (hartree)", fontsize=20)

p2.plot(diatomic_energies["r"][i:j],(diatomic_energies["ccsdt"][i:j] - ccsdt) - (diatomic_energies["hf"][i:j]    - hf), 
       label = "RHF", color='#00a2ff', lw=8, ls='-.')

p2.plot(diatomic_energies["r"][i:j],(diatomic_energies["ccsdt"][i:j] - ccsdt) - (diatomic_energies["dft"][i:j]  - dft), 
       label = "LDA", color='#ff01ff', lw=8, ls='-.')

p2.set_xlim([1, 10])

p2.legend(fontsize=20)

In [ ]:
diatoms = pd.read_csv("diatoms_raw_full.csv")
diatoms.shape

#######################
#Plot of Histogram. Depends on diatomics DF
#######################

mean = np.mean(diatoms.loc[:,"difference"])
print('mean', mean)

sdt = np.std(diatoms.loc[:,"difference"])
print('standard deviation', sdt)

median = np.median(diatoms.loc[:,"difference"])
print('median', median)


fig = plt.figure(figsize=(20,10), dpi=300)

hist = diatoms.loc[:,"difference"].hist(bins=12, color='grey')
plt.title("Histogram of Energy Corrections", fontsize=25)
plt.xlabel("Correction (hartree)", fontsize=25)
plt.ylabel("Count", fontsize=25)

plt.axvline(x=median, lw=10, label='Median', color='#00a2ff')


plt.tick_params(axis='both', which='major', labelsize=25)


plt.legend(fontsize=25)


In [ ]:
##########################
# Individual Predictions. Residuals
##########################
sort_corrections = diatoms.sort_values(by='difference', ).reset_index()
sort_corrections
predicted_correction = np.zeros((len(sort_corrections),2))

for i in (range(0,2800,5)):
    KKR = KernelRidge(0.01, kernel='rbf', gamma=0.0001)
    KKR.fit( np.delete(coulomb_vector_long, (i), axis=0) , diatoms.loc[:,"difference"].drop(i))
    predicted_correction[i,0] = diatoms.loc[i, "difference"]
    predicted_correction[i,1] = KKR.predict( coulomb_vector_long[i:i+1] )
            
predicted_correction

fig,ax = plt.subplots(ncols=2,figsize=(15,5),dpi=300)

p1 = ax[0] 
p2 = ax[1] 

plt.subplots_adjust(wspace = 0.4)

p1.tick_params(labelsize=20)

p1.set_xlabel("Error Correction (hartree)", fontsize=20)
p1.set_ylabel("Predicted Error (hartree)",fontsize=20)

p1.scatter(predicted_correction[:,0],predicted_correction[:,1])
p1.plot(x,y, color='purple')

p1.set_xlim(0.005,1.00)
p1.set_ylim(0.005,1.00)


###################### TWO

p2.tick_params(labelsize=20)


p2.set_xlabel("Error Correction (hartree)", fontsize=20)
p2.set_ylabel("Residual (hartree)", fontsize=20)

plt.scatter(predicted_correction[:,0],predicted_correction[:,1]-predicted_correction[:,0])
plt.plot(xprime,yprime, color='purple',zorder=2)


p2.set_xlim(0.05,1.00)
p2.set_ylim(-0.7,0.7)


In [40]:
################################
#Cube files for Phenyl and Benzyne. 
################################

psi4.set_options({'DFT_SPHERICAL_POINTS': 6,
                  'DFT_RADIAL_POINTS':    5, 
                  'cubeprop_tasks': ['density','orbitals'],
                  'cubeprop_orbitals' : [21],
                  'cubic_grid_overage' : [6.0, 6.0, 6.0],
                  'cubic_grid_spacing' : [.1, 0.1, 0.1],
                  'reference' : 'uhf', 
                  })

#The way cubeprop writes files can only handle one wfn at the time. 
#en_benzyne, wfn_benzyne = psi4.energy("HF/sto-3g", molecule=benzyne, return_wfn=True)
en_phenyl, wfn_phenyl = psi4.energy("HF/sto-3g", molecule=phenyl, return_wfn=True)

benzyne_array, benzyne_info = cubetools.read_cube('Dt.cube')
benzyne_orb, benzyne_orb_info = cubetools.read_cube('Psi_a_21_21-A.cube')

fig, ax = plt.subplots(figsize=(10, 10))

plt.title('HOMO of Phenyl', fontsize=20)
plt.axis('on')
plt.imshow(benzyne_orb[:,60,:],cmap='magma', interpolation='bicubic')
plt.colorbar()
